In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI


# ChatOpenAI 모델을 초기화합니다.
model = ChatOpenAI()
# 대화형 프롬프트를 생성합니다. 이 프롬프트는 시스템 메시지, 이전 대화 내역, 그리고 사용자 입력을 포함합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

In [3]:
# 대화 버퍼 메모리를 생성하고, 메시지 반환 기능을 활성화합니다.
memory = ConversationBufferMemory(
    return_messages=True, memory_key="chat_history")

In [7]:
memory.load_memory_variables({})  # 메모리 변수를 빈 딕셔너리로 초기화합니다.

{'chat_history': []}

In [8]:
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables)
    | itemgetter("chat_history")  # memory_key 와 동일하게 입력합니다.
)

In [9]:
runnable.invoke({"input": "hi!"})

{'input': 'hi!', 'chat_history': []}

In [10]:
chain = runnable | prompt | model

In [11]:
# chain 객체의 invoke 메서드를 사용하여 입력에 대한 응답을 생성합니다.
response = chain.invoke({"input": "만나서 반갑습니다. 제 이름은 테디입니다."})
print(response)  # 생성된 응답을 출력합니다.

content='만나서 반가워요, 테디님! 무엇을 도와드릴까요?' response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 33, 'total_tokens': 63}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-3edd47a4-548a-4b18-9099-b2aaa0b740b4-0' usage_metadata={'input_tokens': 33, 'output_tokens': 30, 'total_tokens': 63}


In [12]:
# 입력된 데이터와 응답 내용을 메모리에 저장합니다.
memory.save_context(
    {"inputs": "만나서 반갑습니다. 제 이름은 테디입니다."}, {"output": response.content}
)

# 저장된 대화기록을 출력합니다.
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='만나서 반갑습니다. 제 이름은 테디입니다.'),
  AIMessage(content='만나서 반가워요, 테디님! 무엇을 도와드릴까요?')]}

In [13]:
# 이름을 기억하고 있는지 추가 질의합니다.
response = chain.invoke({"input": "제 이름이 무엇이었는지 기억하세요?"})
# 답변을 출력합니다.
print(response.content)

네, 당신의 이름은 테디였어요. 어떤 도움이 필요하세요?
